# Introduction

This notebook was created to serve as documentation for the code contained in this repo as well as to provide justification for the design. Specifically, this project was created to solve the following problem:

### Problem:

Create your own dataset of transitions from one of the Atari environments in openai gym. Create a model from this data that can predict the state at the next time step given the state at the current time step and current action as input.

### Deliverables:

- The code and visualization of the results
- Instructions on how to run

### Further Details:

- Free to use whatever library you want to implement
- Please return this back to us in a week if possible
- Be prepared to discuss this on your final interview
- The only thing we would like you to avoid, is copy someone else's solution online (if there are any)


## Solution Overview

The primary constraints for this project were time and computational resources. Given 1 week (approximately 20 hours of actual development time) and access to a single NVIDIA 1080 GTX the priority was producing a complete solution which would need little experimentation and could train quickly (<12 hours). The problem was broken down into the following sequential steps:

1. Train a deep q learning agent
2. Use the agent from step 1 to create a dataset of state transitions
3. Train a convolutional model to learn the dynamics of the system using the dataset from step 2

*Note*: This solution was developed in a virtual environment (see requirements.txt) on a machine running Windows 10 Professional. It is my experience that tensorflow and opencv can behave slightly different across platforms due to the presence of absence of certain libraries like ffmpeg, so please contact me if you encounter issues running on your machine.

## Step 1

The purpose of training the agent before generating the dataset is to ensure that the state space is sampled in a way which is representative of the agent's experience playing the game. The driver to train the agent is contained in heron/train_agent.py and at the top of the file are a list of hyperparameters. The agent is an implementation of the original DeepMind Atari agent and the code is repurposed from https://keras.io/examples/rl/deep_q_network_breakout/. Additional functionality was implemented for checkpointing and saving the code periodically, as well as a series of helper functions available in heron/tools.

For convenience, the result of training for approximately 2 million frames is included in agents/default. To train a new agent, simply set the parameters at the top of the file and run the file.

In [ ]:
Code for generating video

## Step 2

The dataset is created by allowing agent/default to play the game and transitions are sampled according to tools/data_helper.sample_transition. Implementing a method to determine the sampling methodology is an attempt to counter possible bias when generating the dataset, as transitions from early states of the game are likely to be overrepresented if samples are retrieved periodically.

Data is batched according to the parameters batch_size and num_batches and the total number of samples produced will be batch_size * num_batches. Batching the data in this way is not strictly necessary, but I have found that it is very helpful to be able to easily retrieve and view particular examples when debugging and training. Methods for visualizing these examples is available in tools/visualize.

Code for inspecting dataset

## Step 3

Given my design constraints, I decided that this step is where I would try to exercise the most creativity with my solution. Below I have provided a few examples of architectures I thought of to solve the dynamics prediction problem.

## Potential Solutions

1. *Vanilla*: Simply concatenate each action state onto the input layer of a Conv-DeConv network.
    - Pros: 
        - Extremely simple design (saves development time)
        - Can reuse Conv weights from agent
    - Cons: 
        -The action information is of a different type than the visual information of the state. It seems like it would take a very long time for the network to realize that the action is an extremely important part of predicting the next state
        - Having a DeConvNet act on entire image has the extremely high likelihood that the output will not even be a valid game state. This outcome has the consequence of needing a more complex agent to deal with a less precise dynamics model.

2. *UNet with Interior Activation*: Similar architecture to 1, but the action input occurs after the convolutional layers
    - Pros: 
        - The action is directly input alongside the feature maps and so is directly communicated to the DeConv layers, Can reuse Conv weights from agent
    - Cons: 
        - It is possible the ConvNet used to train the agent is too simple for the problem of state prediction for images
        - Having a DeConvNet act on entire image... (same issue as Vanilla)

3. *Attention Based Model*: This network is inspired by the RCNN series of models. The model would essentially be two networks: The first network (what I am calling the attention network) would predcit what areas of the screen will change given a particular action (perhaps by tiling the image and the output of the network would be the tile indices). The second network would be a smaller convolutional model designed to operate on each of the selected tiles to predict how a dynamic tile should be updated given the action. This would run for each of the chosen tiles and the output would be reconstructed from either the original tile or the reconstructed tile based on whether or not the model believed the tile should be updated.
    - Pros: 
        - This model is very explainable, as it captures intermediate data which is easily interpretable by a person. It could be interpreted as a partial success if the agent were able to learn to pay attention to a portion of the screen, even if it wasn't able to correctly discern exactly which pixels were going to change. For an atari game, this information might even be sufficient to train an agent that performs well.
        - This model would likely be fairly small, as the attention network's task is fairly simple, and the second network would be operating on very small images.
        - Because the action layers bypass operating on tiles not selected by the attention network, this model should produce states which are very close (at least in terms of metrics like MSE) to true states of the game.
    - Cons:
        - The network might be tricky to train because of strange dynamics at the interface layer. It would likely need a lot of hyperparameter tuning to deal with partial activations of the attention layer.
        - As far as I know, this architecture is somewhat novel, so its performance would be difficult to benchmark against other solutions

4. *GAN*: Use a generative-adversarial model where the input to the generator is the action
    - Pros: Potentially combats the invalid states problem of solutions 1 and 2
    - Cons: Much more complicated to implement and train


## Design Approach

I decided to implement model 2, since it best meets the trade offs for model complexity and performance given my time constraints (although I'm most excited about trying model 3 at some point).

In [ ]:
Visualizing predictions and metrics to eval performance